In [1]:
import numpy as np
import os
import pandas as pd

# Visualize Final Partition

In [29]:
# Copied/modified from VisTruncOctahedron
def gen_face(faces_str, vertices_str, color_rgb='<red_val, green_val, blue_val>'):
    total_str = ""
    for face_str in faces_str:
        total_str += '// ' + face_str + '\n'
        total_str += 'polygon {' + '\n'

        face = face_str[1:-1].split(',')
        total_str += '\t{},'.format(len(face)) + '\n'

        vtx_str = ""
        for vi in face:
            vtx_str += "<" + vertices_str[int(vi)][1:-1] + ">, "
        vtx_str = vtx_str[:-2] # Remove last comma
        total_str += '\t' + vtx_str + '\n'

        total_str += '\tfinish { ambient 0.42 specular 0.5 }\n'
        total_str += '\tpigment { rgb ' + color_rgb + ' }\n'
        total_str += '}\n\n'
    
    return total_str

## Set Parameters

In [20]:
K = 2
S = 10
honeycomb_type = "grid" # "grid" or bcc"

In [21]:
colors = np.array([[251, 112, 82], [227, 0, 118], [40, 165, 0], [40, 175, 175]]) / 255.
colors_str = ['<' + ','.join(list(color.astype(str))) + '>' for color in colors]

In [22]:
partition_fname = "partition_{}_{}_K{}.csv".format(honeycomb_type, S, K)

## Load Partition

In [23]:
dfpart = pd.read_csv(partition_fname)
dfpart = dfpart.rename(columns={'cell_id': 'ID', 'part': 'Part'})
dfpart = dfpart.set_index('ID')
dfpart.head()

,Part
ID,
0,2
1,2
2,2
3,1
4,1


## Load semicolon-separated version of Voro++ output

In [24]:
in_fname = f"../generate_input/{honeycomb_type}/{honeycomb_type}_{S}.csv"

In [25]:
in_fname_sc = f"../tmp/{honeycomb_type}_{S}_sc.csv"
if not os.path.exists(in_fname_sc):
    raise Exception(f"Missing semicolon-separated version of {in_fname}. Not found at {in_fname_sc}")

In [26]:
df = pd.read_csv(in_fname_sc, sep=';', header=1)
df = df.set_index('ID')
df = df.join(dfpart)
df.head()

,No. Neighbors,Neighbors,Faces,Vertices,Part
ID,,,,,
0,6,100 -3 -5 10 -1 1,"(1,5,7,3) (1,0,4,5) (1,3,2,0) (2,3,7,6) (2,6,4...","(0,0,0) (1,0,0) (0,1,0) (1,1,0) (0,0,1) (1,0,1...",2
1,6,101 -3 0 11 2 -1,"(1,3,5,2) (1,0,4,3) (1,2,6,0) (2,5,7,6) (3,4,7...","(0,0,1) (1,0,1) (1,1,1) (1,0,2) (0,0,2) (1,1,2...",2
2,6,102 -3 1 12 3 -1,"(1,3,5,2) (1,0,4,3) (1,2,6,0) (2,5,7,6) (3,4,7...","(0,0,2) (1,0,2) (1,1,2) (1,0,3) (0,0,3) (1,1,3...",2
3,6,103 -3 2 13 4 -1,"(1,3,5,2) (1,0,4,3) (1,2,6,0) (2,5,7,6) (3,4,7...","(0,0,3) (1,0,3) (1,1,3) (1,0,4) (0,0,4) (1,1,4...",1
4,6,14 104 3 -1 5 -3,"(1,7,2,3) (1,5,6,7) (1,3,0,5) (2,4,0,3) (2,7,6...","(0,0,4) (1,1,4) (0,1,5) (0,1,4) (0,0,5) (1,0,4...",1


In [27]:
parts = sorted(list(np.unique(df['Part'])))

## Generate POV Ray file for each part

In [37]:
for part in parts:
    out_fname = "partition_{}_{}_K{}_{}.pov".format(honeycomb_type, S, K, part)
    with open(out_fname, 'w') as f:
        f.write('#version 3.6;\n')
        f.write('// Right-handed coordinate system in which the z-axis points upwards\n')
        f.write('camera {\n')
        f.write(f"\tlocation <{3*S}, {-6.5*S}, {4*S}>\n")
        f.write('\tsky z\n')
        f.write('\tright -0.22*x*image_width/image_height\n')
        f.write('\tup 0.22*z\n')
        f.write(f"\tlook_at <{S/2}, {S/2}, {S/2}>\n")
        f.write('\t}\n')

        f.write(f"#declare red_val={colors[part - 1][0]};\n");
        f.write(f"#declare green_val={colors[part - 1][1]};\n");
        f.write(f"#declare blue_val={colors[part - 1][2]};\n");

        f.write('// White background\n')
        f.write('background{rgb 1}\n')

        f.write('// Two lights with slightly different colors\n')
        f.write('light_source{<-8,-20,30> color rgb <0.77,0.75,0.75>}\n')
        f.write('light_source{<25,-12,12> color rgb <0.38,0.40,0.40>}\n')
        f.write('')
        
        # Write faces of each cell in part, as polygons
        for idx in df.index:
            if df.loc[idx, 'Part'] == part:
                cell_str = gen_face(df.loc[idx, 'Faces'].split(' '), df.loc[idx, 'Vertices'].split(' '))
                f.write(cell_str)

In [41]:
dirlist = os.listdir('./')
partitions = [file for file in dirlist if (file[:9] == 'partition' and file[-4:] == '.csv')]

6
['partition_bcc_10_K2.csv', 'partition_bcc_10_K25.csv', 'partition_bcc_10_K8.csv', 'partition_grid_10_K2.csv', 'partition_grid_10_K25.csv', 'partition_grid_10_K8.csv']
